#**SberMarket Competition**

Выполнила Бурикова Анна

Ссылка на соревнование https://www.kaggle.com/competitions/sbermarket-internship-competition/overview

В качестве тренировочных данных представляется датасет с историей заказов 20000 пользователей вплоть до даты отсечки, которая разделяет тренировочные и тестовые данные по времени.

train.csv:
user_id - уникальный id пользователя
order_completed_at - дата заказа
cart - список уникальных категорий (category_id), из которых состоял заказ
В качестве прогноза необходимо для каждой пары пользователь-категория из примера сабмита вернуть 1, если категория будет присутствовать в следующем заказе пользователя, или 0 в ином случае. Список категорий для каждого пользователя примере сабмита - это все категории, которые он когда-либо заказывал.

sample_submission.csv:
Пример сабмита. В тест входят не все пользователи из тренировочных данных, так как некоторые из них так ничего и не заказали после даты отсечки.

id - идентификатор строки - состоит из user_id и category_id, разделенных точкой с запятой: f'{user_id};{category_id}'. Из-за особенностей проверяющей системы Kaggle InClass, использовать колонки user_id, category_id в качестве индекса отдельно невозможно
target - 1 или 0 - будет ли данная категория присутствовать в следующем заказе пользователя

**Лучший F1 score - 0.664 получился с использованием идеи предсказания для следующих k категорий Top-k самых частых предыдыдущих, находившихся в заказах пользователя.**

**Также были попытки решить задачу с помщью бинарной классификации пар (user_id, cart_id) и с помощью полносвязной нейронной сети, выдавшей для пользователя по вектору предыдущих покупок вектор предстоящих.**

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [3]:
train_data = pd.read_csv('/content/drive/MyDrive/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

# Подготовка данных

Удалим ненужный столбец, посмотрим на данные

In [4]:
train_data = train_data.drop(columns='Unnamed: 0')
train_data

,user_id,order_completed_at,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198
3,2,2015-03-22 09:25:46,88
4,2,2015-03-22 09:25:46,157
...,...,...,...
3123059,12702,2020-09-03 23:45:45,441
3123060,12702,2020-09-03 23:45:45,92
3123061,12702,2020-09-03 23:45:45,431
3123062,12702,2020-09-03 23:45:45,24


In [5]:
subm = submission.drop(columns='Unnamed: 0')
subm

,id,target
0,0;133,0
1,0;5,1
2,0;10,0
3,0;396,1
4,0;14,0
...,...,...
790444,19998;26,0
790445,19998;31,0
790446,19998;29,1
790447,19998;798,1


Распарсим столбец id в сабмите, а также соберем user_carts, в котором по user id будет список cart_ids, которые надо предсказать

In [6]:
user_id = []
cart = []
def split_id(s):
  s = s.split(';')
  user_id.append(s[0])
  cart.append(s[1])
subm['id'].apply(split_id)

user_carts = {} #по user id список cart_ids которые надо предсказать
for i in range(len(user_id)):
  id = int(user_id[i])
  user_carts[id] = user_carts.get(id, []) + [int(cart[i])]

Будем работать с новым DataFrame, в котором сгруппируем заказы пользователей и для каждого заказа соберем список категорий в заказе

In [13]:
train = train_data.groupby(by=['user_id', 'order_completed_at'])['cart'].apply(list)
train = pd.DataFrame(train)
train

cart
user_id order_completed_at                                                    
0       2020-07-19 09:59:17               [20, 82, 441, 57, 14, 405, 430, 379]
        2020-08-24 08:55:32  [133, 5, 26, 10, 382, 14, 22, 41, 25, 441, 411...
        2020-09-02 07:38:25  [803, 170, 84, 61, 440, 57, 55, 401, 398, 399,...
1       2019-05-08 16:09:41                                               [55]
        2020-01-17 14:44:23                        [82, 798, 86, 421, 204, 55]
...                                                                        ...
19998   2020-09-01 08:12:32                     [398, 57, 84, 61, 415, 6, 420]
        2020-09-02 15:03:23                                 [84, 798, 409, 19]
19999   2020-08-31 18:54:24                                              [326]
        2020-08-31 19:32:08                                              [326]
        2020-08-31 20:08:24                                              [326]

[209406 rows x 1 columns]

# Бинарная кассификация пар (user_id, cart_id): 1 - пользователь купит этот товар, 0 - не купит

Чтобы решать задачу классификации нужно придумать новые фичи и создать датасет, в котором один элемент будет соответствовать конкретной паре user-cart

|user_orders_count| - количество заказов данного пользователя

|user_carts_count| - количество заказов данной категории данного пользователя

|all_carts_count| - количество заказов данной категории всеми пользлвателями

Чтобы обучить классификатор, нужно иметь таргет. Будем составлять его на основе последнего заказа пользлвателя

In [14]:
#датасет
# |user_id| |cart_id| |user_orders_count| |user_carts_count| |all_carts_count| |target|
#n-1 заказ собираем статистику, n-ый заказ - target
#новые фичи
train_df = pd.DataFrame()
train_df_len = len(user_id)
train_df['user_id'] = user_id
train_df['cart_id'] = cart
train_df['user_orders_count'] = np.zeros(train_df_len)
train_df['user_carts_count'] = np.zeros(train_df_len)
train_df['all_carts_count'] = np.zeros(train_df_len)
train_df['target'] = np.zeros(train_df_len)

In [15]:
data = train

In [16]:
unique_carts = np.unique(cart).astype(int)
all_count_carts = {}
for c in unique_carts:
  all_count_carts[c] = 0

In [ ]:
unique_carts = unique_carts.astype(int)
unique_users = np.unique(user_id).astype(int)
unique_carts_max = unique_carts.max()
user_orders_count = {}
user_carts_count = {}
user_carts_target = {}
for id in unique_users:
  user_orders_count[id] = 0
  user_carts_count[id] = [0] * (unique_carts_max + 1)
  user_carts_target[id] = [0] * (unique_carts_max + 1)
for i in range(len(unique_users)):
  id = unique_users[i]
  orders = data.loc[id]
  n = orders.shape[0]
  user_orders_count[id] = n - 1
  for j in range(n - 1):
    categories = orders.iloc[j][0]
    for x in categories:
      if x in unique_carts:
        user_carts_count[id][x] += 1
        all_count_carts[x] += 1
  categories = orders.iloc[n - 1][0]
  for x in categories:
    if x in unique_carts:
      user_carts_target[id][x] = 1

In [18]:
train_df['user_id'] = train_df['user_id'].astype(int)
train_df['cart_id'] = train_df['cart_id'].astype(int)

In [19]:
for i in range(train_df.shape[0]):
  user_id0 = train_df['user_id'][i]
  cart_id0 = train_df['cart_id'][i]
  train_df['user_orders_count'][i] = user_orders_count[user_id0]
  train_df['user_carts_count'][i] = user_carts_count[user_id0][cart_id0]
  train_df['target'][i] = user_carts_target[user_id0][cart_id0]
  train_df['all_carts_count'][i] = all_count_carts[cart_id0]

<ipython-input-19-75515cc3fef1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_orders_count'][i] = user_orders_count[user_id0]
<ipython-input-19-75515cc3fef1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_carts_count'][i] = user_carts_count[user_id0][cart_id0]
<ipython-input-19-75515cc3fef1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['target'][i] = user_carts_target[user_id0][cart_id0]


In [20]:
train_df

,user_id,cart_id,user_orders_count,user_carts_count,all_carts_count,target
0,0,133,2.0,1.0,1014.0,0.0
1,0,5,2.0,1.0,21297.0,0.0
2,0,10,2.0,1.0,10571.0,0.0
3,0,396,2.0,1.0,19264.0,0.0
4,0,14,2.0,2.0,66303.0,0.0
...,...,...,...,...,...,...
790444,19998,26,2.0,1.0,13951.0,0.0
790445,19998,31,2.0,1.0,13458.0,0.0
790446,19998,29,2.0,1.0,19489.0,0.0
790447,19998,798,2.0,0.0,15579.0,1.0


In [21]:
#train_df.to_csv('/content/drive/MyDrive/train_df.csv')

В качестве признаков используем колонки из train_df кроме user_id и cart_id

In [29]:
x1 = train_df['user_orders_count'].to_numpy()
x2 = train_df['user_carts_count'].to_numpy()
x3 = train_df['all_carts_count'].to_numpy()
y = train_df['target'].to_numpy()
y = y.reshape((y.shape[0], 1)).astype(int)
x = np.vstack((x1, x2, x3)).T

Для каких-то классфикаторов может потребоваться нормализация признаков.

In [30]:
'''from sklearn.preprocessing import StandardScaler
scaller = StandardScaler()
scaller.fit(x)
x = scaller.transform(x)'''

'from sklearn.preprocessing import StandardScaler\nscaller = StandardScaler()\nscaller.fit(x)\nx = scaller.transform(x)'

In [31]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)

Для всех классификаторов подбирались гиперпараметры путем некоторого количества запусков. Представлены лучшие варианты.

In [33]:
#!pip install catboost
import catboost
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=10,
    learning_rate=0.05,
    loss_function='CrossEntropy',
)
clf.fit(x_train, y_train,
        verbose=False
)
y_pred = clf.predict(x_test)
from sklearn.metrics import f1_score
f1_score(y_pred=y_pred, y_true=y_test)

0.5257907664544589

**Catboost F1 score - 0.5257907664544589**

Перед запуском логистической регрессии лучш выполнить код с нормализацией признаков

In [34]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)
f1_score(y_pred=y_pred, y_true=y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.15941475632664906

**LogisticRegression F1 score - 0.15941475632664906**

In [35]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(x_train, y_train)
y_pred = gbc.predict(x_test)
f1_score(y_pred=y_pred, y_true=y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5782036593410408

**GradientBoosting F1 score - 0.5782036593410408**

Проверим качество, сравнив предсказанный таргет и таргет сабмита

In [36]:
target = subm['target'].astype(int)
target_pred = gbc.predict(x)
f1_score(y_pred=target_pred, y_true=target)

0.16882084832991356

**Итоговый F1 score для пар из сабмита - 0.16882084832991356**

Хотя во время обучения качество на Catboost или GradientBoosting было неплохим, качество на тесте оказалось не очень хорошим и хотелось бы улучшить модель...

# Выбор для каждого пользователя top k наиболее часто покупаемых товаров, где k - количество товаров для этого пользователя в сабмите

In [38]:
user_topk = {}
user_carts_count1 = {}
for id in unique_users:
  user_topk[id] = [0] * 10
  user_carts_count1[id] = [0] * (unique_carts_max + 1)
for i in range(len(unique_users)):
  id = unique_users[i]
  orders = data.loc[id]
  n = orders.shape[0]
  for j in range(n):
    categories = orders.iloc[j][0]
    for x in categories:
      if x in user_carts[id]:
        user_carts_count1[id][x] += 1

In [39]:
def get_topk(k, a):
  res = []
  a1 = []
  for i in range(len(a)):
    a1.append((a[i], i))
  a1.sort()
  for i in range(len(a) - 1, len(a) - k, -1):
    res.append(a1[i][1])
  return res

In [40]:
for x in user_carts_count1.keys():
  user_topk[x] = get_topk(len(user_carts[x]), user_carts_count1[x])

In [41]:
target_topk = []
for i in range(train_df.shape[0]):
  user_id0 = train_df['user_id'][i]
  cart_id0 = train_df['cart_id'][i]
  if cart_id0 in user_topk[user_id0]:
    target_topk.append(1)
  else:
    target_topk.append(0)

In [42]:
f1_score(y_pred=target_topk, y_true=target)

0.663601155064316

**Итоговый F1 score - 0.663601155064316**

Несмотря на простоту, этот вариант оказался заметно лучше обученных моделей.

Замечание к обобщению этого метода - может оказаться необходимо предсказать для k категрой для конкретного пользователя, купит ли он товар, а в истории покупок пользователя будет менее k покупок различных категорий. Тогда данная реализация выдаст частично случайные ответы. Этого можно избежать, выдавая, например, наиболее часто встречающиеся категории среди всех пользователей.

# Совсем плохо сработавший вариант

Одна из идей - описывать пользователя вектором размера максимального id категории товара и ставить в позиции, соответствующей категории число её заказов. Таргет - вектор такого же размера с 1 в позициях, которые приобрел пользователь.

Это было решено реализовать с помощью полносвязной нейронной сети с тремя линейными слоями.

Но сеть выдавала свосем не имеющие смысл предсказания и долго (час) обучалась на CPU, поэтому этот вариант не стал рассматриваться как возможный.

In [ ]:
train = train_data.groupby(by=['user_id', 'order_completed_at'])['cart'].apply(list)
train = pd.DataFrame(train)
train

cart
user_id order_completed_at                                                    
0       2020-07-19 09:59:17               [20, 82, 441, 57, 14, 405, 430, 379]
        2020-08-24 08:55:32  [133, 5, 26, 10, 382, 14, 22, 41, 25, 441, 411...
        2020-09-02 07:38:25  [803, 170, 84, 61, 440, 57, 55, 401, 398, 399,...
1       2019-05-08 16:09:41                                               [55]
        2020-01-17 14:44:23                        [82, 798, 86, 421, 204, 55]
...                                                                        ...
19998   2020-09-01 08:12:32                     [398, 57, 84, 61, 415, 6, 420]
        2020-09-02 15:03:23                                 [84, 798, 409, 19]
19999   2020-08-31 18:54:24                                              [326]
        2020-08-31 19:32:08                                              [326]
        2020-08-31 20:08:24                                              [326]

[209406 rows x 1 columns]

In [ ]:
subm = submission.drop(columns=['Unnamed: 0'])
subm['user_id'] = np.zeros(subm.shape[0])
subm['cart'] = np.zeros(subm.shape[0])
subm

,id,target,user_id,cart
0,0;133,0,0.0,0.0
1,0;5,1,0.0,0.0
2,0;10,0,0.0,0.0
3,0;396,1,0.0,0.0
4,0;14,0,0.0,0.0
...,...,...,...,...
790444,19998;26,0,0.0,0.0
790445,19998;31,0,0.0,0.0
790446,19998;29,1,0.0,0.0
790447,19998;798,1,0.0,0.0


In [ ]:
user_ids = []
carts = []
def f(s):
  s = s.split(';')
  user_ids.append(s[0])
  carts.append(s[1])
subm['id'].apply(f)

In [ ]:
subm['user_id'] = user_ids
subm['cart'] = carts
subm.drop(columns=['id'])

,target,user_id,cart
0,0,0,133
1,1,0,5
2,0,0,10
3,1,0,396
4,0,0,14
...,...,...,...
790444,0,19998,26
790445,0,19998,31
790446,1,19998,29
790447,1,19998,798


In [ ]:
subm_cart = pd.DataFrame(subm.groupby('user_id')['cart'].apply(list))
subm_target = pd.DataFrame(subm.groupby('user_id')['target'].apply(list))
subm_target['cart'] = subm_cart['cart']
target_ids = np.unique(subm['user_id'])

In [ ]:
train

cart
user_id order_completed_at                                                    
0       2020-07-19 09:59:17               [20, 82, 441, 57, 14, 405, 430, 379]
        2020-08-24 08:55:32  [133, 5, 26, 10, 382, 14, 22, 41, 25, 441, 411...
        2020-09-02 07:38:25  [803, 170, 84, 61, 440, 57, 55, 401, 398, 399,...
1       2019-05-08 16:09:41                                               [55]
        2020-01-17 14:44:23                        [82, 798, 86, 421, 204, 55]
...                                                                        ...
19998   2020-09-01 08:12:32                     [398, 57, 84, 61, 415, 6, 420]
        2020-09-02 15:03:23                                 [84, 798, 409, 19]
19999   2020-08-31 18:54:24                                              [326]
        2020-08-31 19:32:08                                              [326]
        2020-08-31 20:08:24                                              [326]

[209406 rows x 1 columns]

In [ ]:
df = train
user_ids = target_ids.astype(int)
x_train = np.zeros((target_ids.shape[0], np.unique(train_data['cart']).max() + 1))
y_train = np.zeros((target_ids.shape[0], np.unique(train_data['cart']).max() + 1))
for i in range(user_ids.shape[0]):
  orders = df.loc[user_ids[i]]
  n = orders.shape[0]
  for j in range(n - 1):
    categories = orders.iloc[j][0]
    for x in categories:
      x_train[i][x] += 1
  categories = orders.iloc[n - 1][0]
  for x in categories:
    y_train[i][x] = 1

In [ ]:
carts = np.unique(train_data['cart']).max() + 1
carts

881

In [ ]:
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(881, carts + 500)
        self.relu1 = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(carts + 500)
        self.linear2 = nn.Linear(carts + 500, carts * 2)
        self.relu2 = nn.ReLU()
        self.batchnorm2 = nn.BatchNorm1d(carts * 2)
        self.linear3 = nn.Linear(carts * 2, carts)
        self.s = nn.Softmax(dim=1)

    def forward(self, x1):
        x1 = self.linear1(x1)
        x1 = self.relu1(x1)
        x1 = self.batchnorm1(x1)
        x1 = self.relu2(self.linear2(x1))
        x1 = self.batchnorm2(x1)
        x1 = self.s(self.linear3(x1))
        return x1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNet().to(device)
# функция потерь
loss_fn = torch.nn.CrossEntropyLoss()

# оптимизатор
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
x = torch.tensor(x_train, dtype=torch.float32)
y = torch.tensor(y_train, dtype=torch.float32)

In [ ]:
def train1(model, loss_fn, optimizer, n_epoch=6):

    model.train(True)

    # цикл обучения сети
    for epoch in tqdm(range(n_epoch)):

        for i in range(0, x.shape[0], 2):

            # так получаем текущий батч картинок и ответов к ним
            X_batch, y_batch = torch.cat((x[i].unsqueeze(0), x[i + 1].unsqueeze(0))), torch.cat((y[i].unsqueeze(0), y[i + 1].unsqueeze(0)))

            # forward pass (получение ответов сети на батч картинок)
            logits = model(X_batch.to(device))

            # вычисление лосса от выданных сетью ответов и правильных ответов на батч
            loss = loss_fn(logits, y_batch.to(device))

            # каждые 50 итераций будем выводить лосс на текущем батче
            if i % 50 == 0:
                print(loss.item())

            optimizer.zero_grad() # обнуляем значения градиентов оптимизаторв
            loss.backward() # backpropagation (вычисление градиентов)
            optimizer.step() # обновление весов сети


    return model

In [ ]:
model = train1(model, loss_fn, optimizer, n_epoch=4)

In [ ]:
def evaluate(model, loss_fn):

    y_pred_list = []
    y_true_list = []
    losses = []

    # проходимся по батчам даталоадера
    for i in range(x.shape[0] // 5):

        # так получаем текущий батч
        X_batch, y_batch = torch.cat((x[i].unsqueeze(0), x[i + 1].unsqueeze(0))), torch.cat((y[i].unsqueeze(0), y[i + 1].unsqueeze(0)))
        # выключаем подсчет любых градиентов
        with torch.no_grad():

            # получаем ответы сети на батч
            logits = model(X_batch.to(device))

            # вычисляем значение лосс-функции на батче
            loss = loss_fn(logits, y_batch.to(device))
            loss = loss.item()

            # сохраняем лосс на текущем батче в массив
            losses.append(loss)

        # сохраняем в массивы правильные ответы на текущий батч
        # и ответы сети на текущий батч
        y_pred_list.extend(logits.cpu().numpy())
        y_true_list.extend(y_batch.numpy())
        #print('pred', y_pred, 'batch', y_batch)

    # считаем accuracy между ответам сети и правильными ответами
    #accuracy = accuracy_score(y_pred_list, y_true_list)

    return y_pred_list, y_true_list, np.mean(losses)

In [ ]:
y1, y2, a = evaluate(model, loss_fn)

In [ ]:
np.where(y1[9] > 0.0000000000000000000000000001)

(array([57]),)

In [ ]:
np.where(y2[9] > 0)

(array([ 89,  90, 170]),)

# Выводы

Выбор top-k частых категорий оказался лучшим способом.

Но, возможно, добавив больше признаков в варианте с классификацией пар, его можно было бы улучшить. Взаимосвзь между категориями могла бы дать много знаний о том, какого типа товары покупает человек. Например, имея словесное описание категорий, можно было бы предлагать пользователю товар близкой по смыслу категории.